In [ ]:

import os
import base64
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment = "gpt-4o-mini"

search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
search_index = "margie-fileupload-index"

show_citations = True

# Initialize Azure OpenAI client with key-based authentication
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2025-01-01-preview",
)

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

# Prepare the chat prompt
chat_prompt = [
    { "role" : "system", "content" : "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다." },
    { "role" : "user", "content" : "마지 여행사에 대해서 알려줄래?" }
]

# Include speech result if speech is enabled
messages = chat_prompt

# Generate the completion
completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    max_tokens=6553,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
    extra_body={
      "data_sources": [{
          "type": "azure_search",
          "parameters": {
            "endpoint": f"{search_endpoint}",
            "index_name": search_index,
            "semantic_configuration": "default",
            "query_type": "simple", #"semantic"
            "fields_mapping": {},
            "in_scope": True,
            # "role_information": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다.",
            "filter": None,
            "strictness": 3,
            "top_n_documents": 5,
            "authentication": {
              "type": "api_key",
              "key": f"{search_key}"
            },
            "embedding_dependency": {
              "type": "deployment_name",
              "deployment_name": "text-embedding-ada-002"
            }
          }
        }]
    }
)

# print(completion.to_json())
print(completion.choices[0].message.content)

if show_citations :
    print("\nCitations : ")

    for c in range(len(completion.choices[0].message.context["citations"])) :

        citation_title = completion.choices[0].message.context["citations"][c]["title"]
        citation_content = completion.choices[0].message.context["citations"][c]["content"][:30]

        print(f"Doc no.{c+1}")
        print(" Title: " + citation_title + "\n Content: " + citation_content + "\n")
    

마지 여행사는 국내적인 여행 서비스와 현지 전문 지식을 제공하는 회사입니다. 이 회사는 다양한 여행 패키지를 통해 고객에게 여행의 즐거움을 선사하며, 여러 유명 도시와 관광지를 포함한 여러 여행지를 다룹니다.

주요 서비스로는 다음과 같은 것들이 있습니다:
- 맞춤형 여행 패키지 제공
- 전문 가이드와 함께하는 투어
- 항공권 및 숙소 예약 서비스

마지 여행사의 CEO는 Marjorie Long이며, CFO는 Logan Reid, CTO는 Emma Luffman입니다 . 마지 여행사는 고객의 편안하고 즐거운 여행을 위해 최선을 다하고 있습니다 [doc1][doc2][doc3][doc4][doc5]. 

자세한 정보는 마지 여행사의 웹사이트인 www.margiestravel.com에서 확인할 수 있습니다 [doc1].

Citations : 
Doc no.1
 Title: Margies Travel Company Info_ko.pdf
 Content: 
가고 싶은 곳이 어디든지, 
Margie’s Trav

Doc no.2
 Title: San Francisco Brochure_ko.pdf
 Content: 
Margie’s Travel Presents…

샌프

Doc no.3
 Title: Dubai Brochure_ko.pdf
 Content: 
Margie’s Travel Presents…

두바

Doc no.4
 Title: Las Vegas Brochure_ko.pdf
 Content: 
Margie’s Travel Presents…

라스

Doc no.5
 Title: London Brochure_ko.pdf
 Content: 
Margie’s Travel Presents…

런던



In [ ]:
# 함수화 해보기
# 그냥 함수명 지정하고 전부 안에 집어넣어도 잘 작동

import os
import base64
from openai import AzureOpenAI
from dotenv import load_dotenv

def openai_rag_with_embedding() :

    load_dotenv()

    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    subscription_key = os.getenv("AZURE_OPENAI_API_KEY")
    deployment = "gpt-4o-mini"

    search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
    search_key = os.getenv("AZURE_SEARCH_API_KEY")
    search_index = "margie-rag"

    show_citations = True

    # Initialize Azure OpenAI client with key-based authentication
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=subscription_key,
        api_version="2025-01-01-preview",
    )

    # IMAGE_PATH = "YOUR_IMAGE_PATH"
    # encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

    # Prepare the chat prompt
    chat_prompt = [
        { "role" : "system", "content" : "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다." },
        { "role" : "user", "content" : "마지 여행사에 대해서 알려줄래?" }
    ]

    # Include speech result if speech is enabled
    messages = chat_prompt

    # Generate the completion
    completion = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=6553,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
        extra_body={
        "data_sources": [{
            "type": "azure_search",
            "parameters": {
                "endpoint": f"{search_endpoint}",
                "index_name": "margie-rag",
                "semantic_configuration": "margie-rag-semantic-configuration",
                "query_type": "vector_semantic_hybrid",
                "fields_mapping": {},
                "in_scope": True,
                # "role_information": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다.",
                "filter": None,
                "strictness": 3,
                "top_n_documents": 5,
                "authentication": {
                "type": "api_key",
                "key": f"{search_key}"
                },
                "embedding_dependency": {
                "type": "deployment_name",
                "deployment_name": "text-embedding-ada-002"
                }
            }
            }]
        }
    )

    # print(completion.to_json())
    print(completion.choices[0].message.content)

    if show_citations :
        print("\nCitations : ")

        for c in range(len(completion.choices[0].message.context["citations"])) :

            citation_title = completion.choices[0].message.context["citations"][c]["title"]
            citation_content = completion.choices[0].message.context["citations"][c]["content"][:30]

            print(f"Doc no.{c+1}")
            print(" Title: " + citation_title + "\n Content: " + citation_content + "\n")
    

In [4]:
openai_rag_with_embedding()

마지 여행사는 다양한 여행 서비스를 제공하는 회사로, 국내외 여행 패키지와 맞춤형 여행 계획을 전문으로 합니다. 회사의 주요 서비스에는 다음과 같은 내용이 포함됩니다:

- **여행 패키지**: 마지 여행사는 다수의 인기 여행지를 대상으로 한 패키지 투어를 운영합니다. 예를 들어, 샌프란시스코, 두바이, 뉴욕, 런던 등 다양한 도시를 포함한 여행 상품을 제공합니다 [doc3].
- **맞춤형 여행 계획**: 고객의 요구에 맞춘 맞춤형 여행 계획을 수립하여 개인적인 여행 경험을 제공합니다 .
- **전문가 팀**: 마지 여행사는 경험이 풍부한 전문가 팀을 보유하고 있으며, 고객에게 최상의 서비스와 지원을 제공합니다. 주요 팀원으로는 CEO 마조리 롱, CFO 로건 리드, CTO 엠마 러프먼 등이 있습니다 [doc1].

마지 여행사는 고객이 원하는 여행 스타일과 예산에 맞춰 다양한 옵션을 제공하며, 고객의 여행 경험을 더욱 풍부하게 만들기 위해 노력하고 있습니다. 추가 정보는 회사 웹사이트인 [www.margiestravel.com](http://www.margiestravel.com)에서 확인할 수 있습니다 [doc1].

Citations : 
Doc no.1
 Title: Margies Travel Company Info_ko.pdf
 Content: 가고 싶은 곳이 어디든지, 
Margie’s Trave

Doc no.2
 Title: San Francisco Brochure_ko.pdf
 Content: Margie’s Travel Presents…

샌프란

Doc no.3
 Title: Dubai Brochure_ko.pdf
 Content: Margie’s Travel Presents…

두바이

Doc no.4
 Title: New York Brochure_ko.pdf
 Content: Margie’s Travel Presents…

뉴욕,

Doc no.5
 Title: London Brochure_ko.pdf
 Content: Margie’s